In [ ]:
import pandas as pd
import time
from Functions import xbrl_dict,filings_collector
import datetime as dt
import requests
import xml.etree.ElementTree as ET
from pymongo import MongoClient

In [ ]:
ticker = input('ticker?').lower()

In [3]:
# Access page
all_filings_df = filings_collector(ticker)
all_filings_df.to_csv(f'../data/all_{ticker}_filings.csv', index=False)
all_filings_path = f'../data/all_{ticker}_filings.csv'

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [ ]:
all_filings_path = f'../data/all_{ticker}_filings.csv'
xml_filings_df = pd.read_csv(all_filings_path)
xml_filings_df = xml_filings_df.dropna(subset=['xbrl_link'])
xml_filings_df = xml_filings_df.loc[xml_filings_df['filings'] == '10-K']
xml_filings_df = xml_filings_df.reset_index(drop=True)
xml_filings_df.head(100)

In [ ]:
# lxml_ten_df = xml_filings_df.loc[xml_filings_df['filings'] == '10-Q']
filings_list = []
for i in range(0,len(xml_filings_df)):
    lxml_link = xml_filings_df['xbrl_link'][i]

    lxml_dict = xbrl_dict(lxml_link)
    
    filings_list.append(lxml_dict)


In [ ]:
all_data_df = pd.DataFrame(filings_list)
columns = all_data_df.columns.tolist()

In [ ]:
establising_headers = ['company_name', 'CIK','doc_date',
              'doc_year','doc_period','common_shares_outstanding',
              'amendment_flag']
for header in establising_headers:
    index = columns.index(header)
    columns.pop(index)

ordered_columns = establising_headers+columns


In [ ]:
all_data_df = all_data_df[ordered_columns]
all_data_df.tail(20)

In [ ]:
all_years = []
for year in list(all_data_df['doc_year']):
    if year not in all_years:
         all_years.append(year)
counter = 1            
for index, row in all_data_df.iterrows():
#     year = row['doc_year']
#     period = row['doc_period']
    counter += 1
    print(row)
    print(counter)
    
    
    

In [ ]:
all_data_df.to_csv(f'../data/{ticker}_financials.csv', index=False)

In [ ]:
# Mongo DB connection
client = MongoClient('mongodb://localhost:27017')
db = client.Company_Info
collection = db.SEC_filings

for filing in filings_list:
    collection.insert(filing)


In [ ]:
# uris = []
# all_names = []
# for i in range(0,len(xml_filings_df)):
#     lxml_link = xml_filings_df['xbrl_link'][i]
#     xbrl_resp = requests.get(lxml_link)
#     time.sleep(.1)
#     xbrl_str = xbrl_resp.text
#     root = ET.fromstring(xbrl_str)

#     #     defining namespace prefixes
#     uri_list = []
#     names = []
#     for child in root:
#         uri_pos = child.tag.index('}')
#         uri = ''.join(child.tag[1:uri_pos])
#         name = ''.join(child.tag[int(uri_pos+1):len(child.tag)])
        
#         if uri not in uri_list:
#             uri_list.append(uri)
        
#         if uri[0:18] == 'http://xbrl.us/dei':
#             names.append(f'{name}: {child.text}')
#     uris.append(uri_list)
#     all_names.append(names)

In [ ]:
lxml_link = xml_filings_df['doc_link'][21]
print(lxml_link)